In [1]:
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer

import matplotlib.pyplot as plt

tf.get_logger().setLevel('ERROR')

In [62]:
print("Available models")
directories_in_curdir = sorted(list(filter(os.path.isdir, os.listdir(os.curdir))))
available_models = [i for i in directories_in_curdir if i.startswith('model_')]
print(available_models)

Available models
['model_albert_en_base_20221121-143742', 'model_small_bert_bert_en_uncased_L-2_H-128_A-2_20221121-091025', 'model_small_bert_bert_en_uncased_L-2_H-128_A-2_20221121-135444', 'model_small_bert_bert_en_uncased_L-4_H-512_A-8_20221120-232512']


In [63]:
reloaded_model = tf.saved_model.load("model_albert_en_base_20221121-143742")

In [64]:
def is_funny(inputs, results):
  result_for_printing = \
    [f'{inputs[i]:<30}\n FUNNY SCORE: {results[i][0]:.6f}\n'
                         for i in range(len(inputs))]
  print(*result_for_printing, sep='\n')
  print()


examples = [
    'My wife told me to stop impersonating a flamingo. I had to put my foot down.',
    'I went to buy some camo pants but couldn’t find any.',
    'I failed math so many times at school, I can’t even count.',
    'Did you know that the boomerang is Australias biggest export? Its also their biggest import.',
    'Don’t you hate it when someone answers their own questions? I do.',
    'Germany is the second most populous country in Europe after Russia.',
    "Cost of living calculations are also used to compare the cost of maintaining a certain standard of living in different geographic areas.",
    "Along with Hong Kong, South Korea, and Taiwan, Singapore is one of the Four Asian Tigers, and has surpassed its peers in terms of Gross Domestic Product (GDP) per capita.",
    "Nowadays Italy is the fifth most visited country in international tourism.",
    "This TF Hub model uses the implementation of BERT from the TensorFlow Models repository on GitHub."
]

reloaded_results = tf.sigmoid(reloaded_model(tf.constant(examples)))

is_funny(examples, reloaded_results)

My wife told me to stop impersonating a flamingo. I had to put my foot down.
 FUNNY SCORE: 0.994432

I went to buy some camo pants but couldn’t find any.
 FUNNY SCORE: 0.993870

I failed math so many times at school, I can’t even count.
 FUNNY SCORE: 0.978977

Did you know that the boomerang is Australias biggest export? Its also their biggest import.
 FUNNY SCORE: 0.989559

Don’t you hate it when someone answers their own questions? I do.
 FUNNY SCORE: 0.989809

Germany is the second most populous country in Europe after Russia.
 FUNNY SCORE: 0.100006

Cost of living calculations are also used to compare the cost of maintaining a certain standard of living in different geographic areas.
 FUNNY SCORE: 0.714653

Along with Hong Kong, South Korea, and Taiwan, Singapore is one of the Four Asian Tigers, and has surpassed its peers in terms of Gross Domestic Product (GDP) per capita.
 FUNNY SCORE: 0.071859

Nowadays Italy is the fifth most visited country in international tourism.
 FUNNY SC